In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,39672.75,0.75,0.20,0.13,0.00,0.00,-0.00,-0.00
3,0.75,8744.73,-0.09,0.05,-0.00,0.00,-0.00,0.00
4,0.20,-0.09,1655.31,0.01,0.00,-0.00,0.00,-0.00
5,0.13,0.05,0.01,279.75,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,39.93,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,5.21,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.64,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00002,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00002,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00002,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00004,0.00002,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.1229623911818138

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.96950968e+08 9.36409439e+06 3.26239172e+05 8.94511860e+03
 1.70901792e+02 2.58706588e+00 2.98782761e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999437,-0.033009,-0.005912,-0.000995,-0.000142,-0.000019,-0.000002,-2.604396e-07
1,0.032791,0.998906,-0.032893,-0.005325,-0.000760,-0.000099,-0.000013,-1.233597e-06
2,0.006946,0.032473,0.998826,-0.034923,-0.004817,-0.000630,-0.000077,-8.600116e-06
3,0.001404,0.006386,0.034522,0.998703,-0.036536,-0.004644,-0.000574,-6.276462e-05
4,0.000250,0.001138,0.006009,0.036099,0.998333,-0.044284,-0.005355,-5.816177e-04
5,0.000043,0.000197,0.001044,0.006160,0.043656,0.997031,-0.062762,-6.667372e-03
6,0.000008,0.000036,0.000191,0.001132,0.007938,0.061372,0.992145,-1.087079e-01
7,0.000002,0.000008,0.000042,0.000249,0.001743,0.013373,0.108075,9.940512e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005629253640178078,
 0.0010936793813186574,
 0.0011736514164911727,
 0.0012965087348695281,
 0.0016667065778641632,
 0.0029686201715743987,
 0.007855005373833324,
 0.005948794665746493]